In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

# 设置随机种子以确保结果可复现
torch.manual_seed(42)
np.random.seed(42)

# 定义参数
VOCAB_SIZE = 1000
EMBED_DIM = 32
NUM_CLASSES = 2
BATCH_SIZE = 16
NUM_EPOCHS = 10

In [3]:
class SimpleDataset(Dataset):
    def __init__(self, num_samples=1000):
        self.data = []
        for _ in range(num_samples):
            sentence = torch.randint(0, VOCAB_SIZE, (20,))  # 20个词的句子
            label = torch.randint(0, NUM_CLASSES, (1,))
            self.data.append((sentence, label))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# 创建数据集和数据加载器
dataset = SimpleDataset()
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [5]:
class SelfAttention(nn.Module):
    def __init__(self, embed_dim):
        super(SelfAttention, self).__init__()
        self.query = nn.Linear(embed_dim, embed_dim)
        self.key = nn.Linear(embed_dim, embed_dim)
        self.value = nn.Linear(embed_dim, embed_dim)
        
    def forward(self, x):
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)
        
        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / (EMBED_DIM ** 0.5)
        attention_weights = torch.softmax(attention_scores, dim=-1)
        
        return torch.matmul(attention_weights, V)

class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super(TextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.attention = SelfAttention(embed_dim)
        self.fc = nn.Linear(embed_dim, num_classes)
        
    def forward(self, x):
        x = self.embedding(x)
        x = self.attention(x)
        x = x.mean(dim=1)  # 对所有词向量取平均
        return self.fc(x)

In [6]:
def train_model(model, dataloader, num_epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())
    
    for epoch in range(num_epochs):
        total_loss = 0
        correct = 0
        total = 0
        
        for batch_x, batch_y in dataloader:
            optimizer.zero_grad()
            outputs = model(batch_x)
            loss = criterion(outputs, batch_y.squeeze())
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y.squeeze()).sum().item()
        
        accuracy = 100 * correct / total
        print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {total_loss:.4f}, Accuracy: {accuracy:.2f}%')

# 创建模型并开始训练
model = TextClassifier(VOCAB_SIZE, EMBED_DIM, NUM_CLASSES)
train_model(model, dataloader, NUM_EPOCHS)

Epoch [1/10], Loss: 43.9184, Accuracy: 50.20%
Epoch [2/10], Loss: 43.5044, Accuracy: 53.70%
Epoch [3/10], Loss: 43.1205, Accuracy: 55.40%
Epoch [4/10], Loss: 42.6211, Accuracy: 59.10%
Epoch [5/10], Loss: 41.4798, Accuracy: 62.90%
Epoch [6/10], Loss: 39.8191, Accuracy: 65.90%
Epoch [7/10], Loss: 37.3090, Accuracy: 69.40%
Epoch [8/10], Loss: 34.3026, Accuracy: 73.30%
Epoch [9/10], Loss: 30.6382, Accuracy: 77.60%
Epoch [10/10], Loss: 26.6460, Accuracy: 80.90%


In [19]:
def predict(model, text):
    # 将模型设置为评估模式
    model.eval()
    
    # 将文本转换为模型可以处理的格式
    # 这里假设我们有一个函数可以将文本转换为词索引
    input_tensor = text_to_tensor(text).unsqueeze(0)  # 添加批次维度
    
    # 进行预测
    with torch.no_grad():  # 不计算梯度
        output = model(input_tensor)
    
    probabilities = torch.softmax(output, dim=1)
    # 获取预测结果
    confidence, predicted = torch.max(output, 1)
        # 获取具体的预测值
    predicted_value = predicted.item()
    confidence_value = confidence.item()
    all_probs = probabilities.squeeze().tolist()
    labels = ["负面", "正面"]
    predicted_label = labels[predicted_value]
    print(probabilities)
    print(confidence)
    print(predicted)
    print(predicted_value)
    print(confidence_value)
    print(all_probs)
    print(predicted_label)
    # 将预测结果转换为标签
    label = "正面" if predicted.item() == 1 else "负面"
    
    return label

# 假设的文本到张量转换函数
def text_to_tensor(text):
    # 这里应该实现将文本转换为词索引的逻辑
    # 为了简单起见,我们这里只是随机生成一个张量
    return torch.randint(0, VOCAB_SIZE, (20,))

# 使用示例
# model = TextClassifier(VOCAB_SIZE, EMBED_DIM, NUM_CLASSES)
# 假设模型已经训练好了
# model.load_state_dict(torch.load('trained_model.pth'))

In [20]:
# 进行预测
sample_text = "I hate you."
prediction = predict(model, sample_text)
print(f"评论 '{sample_text}' 的情感预测结果是: {prediction}")

tensor([[0.2933, 0.7067]])
tensor([0.5580])
tensor([1])
1
0.5579589009284973
[0.29331257939338684, 0.7066874504089355]
正面
评论 'I hate you.' 的情感预测结果是: 正面
